In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [4]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [5]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
117047,Thiago Alcántara do Nascimento,4,3,90,Liverpool,Leeds,1.953121,0.556956,False,7,...,0.3,3.0,1.0,58.0,61.0,95.1,47.0,0.0,0.0,1
117048,Thiago Emiliano da Silva,4,2,90,Chelsea,Aston Villa,1.959783,0.906719,True,6,...,0.1,1.0,0.0,58.0,66.0,87.9,46.0,0.0,0.0,1
117049,Thomas Partey,4,3,28,Arsenal,Norwich,1.218063,0.420937,True,0,...,0.0,0.0,0.0,10.0,11.0,90.9,8.0,0.0,0.0,1
117050,Timothy Castagne,4,2,90,Leicester City,Manchester City,1.221616,2.309546,True,2,...,0.0,2.0,0.0,26.0,30.0,86.7,22.0,0.0,0.0,1
117051,Timo Werner,4,4,8,Chelsea,Aston Villa,1.959783,0.906719,True,1,...,0.0,0.0,0.0,1.0,1.0,100.0,1.0,0.0,0.0,1
117052,Tim Krul,4,1,90,Norwich,Arsenal,0.420937,1.218063,False,3,...,0.0,1.0,0.0,30.0,50.0,60.0,24.0,0.0,0.0,1
117053,Tino Livramento,4,2,90,Southampton,West Ham United,0.535859,0.787799,True,6,...,0.1,2.0,0.0,35.0,42.0,83.3,37.0,1.0,2.0,1
117054,Tobi Omole,4,2,0,Tottenham Hotspur,Crystal Palace,1.547840,0.531751,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
117055,Toby Alderweireld,4,2,0,Tottenham Hotspur,Crystal Palace,1.547840,0.531751,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
117056,Todd Cantwell,4,3,28,Norwich,Arsenal,0.420937,1.218063,False,1,...,0.0,0.0,0.0,9.0,15.0,60.0,12.0,0.0,0.0,1


In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

5

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
df_latest.head(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Aaron Connolly,5,4,0,Brighton and Hove Albion,Leicester City,0.552039,1.224008,True,0,...,9659,1,2,0.0,-2548,301,2849,0,2021-09-19T13:00:00Z,2122
1,Aaron Cresswell,5,2,90,West Ham United,Manchester United,0.786004,2.085443,True,1,...,1265797,2,1,0.0,-69118,53937,123055,0,2021-09-19T13:00:00Z,2122
2,Aaron Lennon,5,3,0,Burnley,Arsenal,0.323302,1.220448,True,0,...,859,1,0,0.0,89,229,140,0,2021-09-18T14:00:00Z,2122
3,Aaron Ramsdale,5,1,90,Arsenal,Burnley,1.220448,0.323302,False,9,...,77277,1,0,0.0,52488,65020,12532,0,2021-09-18T14:00:00Z,2122
4,Aaron Ramsey,5,3,0,Aston Villa,Everton,0.908494,0.965122,True,0,...,16181,0,3,0.0,138,1995,1857,0,2021-09-18T16:30:00Z,2122
5,Aaron Wan-Bissaka,5,2,90,Manchester United,West Ham United,2.085443,0.786004,False,2,...,542701,2,1,21.0,-79631,8006,87637,0,2021-09-19T13:00:00Z,2122
6,Abdoulaye Doucouré,5,3,90,Everton,Aston Villa,0.965122,0.908494,False,2,...,237412,0,3,0.0,83551,112556,29005,0,2021-09-18T16:30:00Z,2122
7,Adama Traoré,5,3,90,Wolverhampton Wanderers,Brentford,0.870223,0.370362,True,2,...,369081,2,0,25.0,45561,95236,49675,0,2021-09-18T11:30:00Z,2122
8,Adam Armstrong,5,4,90,Southampton,Manchester City,0.536908,2.336319,False,2,...,249612,0,0,71.0,-28248,35537,63785,0,2021-09-18T14:00:00Z,2122
9,Adam Forshaw,5,3,0,Leeds,Newcastle United,0.562274,0.537131,False,0,...,1246,1,1,0.0,-41,227,268,0,2021-09-17T19:00:00Z,2122


In [10]:
df_latest.shape

(606, 34)

In [11]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [12]:
last_play_proba.head()

,player,team,play_proba
0,Joseph Willock,Newcastle United,1.0
1,Jonjo Shelvey,Newcastle United,0.0
2,Matt Ritchie,Newcastle United,1.0
3,Ciaran Clark,Newcastle United,1.0
4,Karl Darlow,Newcastle United,1.0


In [13]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [14]:
df_latest.drop_duplicates(inplace=True)

In [15]:
df_train.drop_duplicates(inplace=True)

In [16]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [17]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [18]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [19]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
Chelsea
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Antonio Rüdiger
Marcos Alonso
Jorginho
Mateo Kovačić
César Azpilicueta
Andreas Christensen
Romelu Lukaku
Edouard Mendy
Mason Mount
Kai Havertz
Thiago Silva
Trevoh Chalobah
Reece James
Timo Werner
N'Golo Kanté
Hakim Ziyech
Kepa Arrizabalaga
Callum Hudson-Odoi
Christian Pulisic
Saúl Ñíguez
Emerson Palmieri
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Alisson
Virgil van Dijk
Sadio Mané
Mohamed Salah
Trent Alexander-Arnold
Joël Matip
Diogo Jota
Fabinho
Jordan Henderson
Kostas Tsimikas
Harvey Elliott
Naby Keïta
James Milner
Thiago Alcántara
Andrew Robertson
Ibrahima Konaté
Roberto Firmino
Alex Oxlade-Chamberlain
Curtis Jones
Divock Origi
Joe Gomez
Manchester Utd
https://fbref.com/en/squads/19538871/Manchester-United-Stats
David de Gea
Harry Maguire
Luke Shaw
Aaron Wan-Bissaka
Bruno Fernandes
Mason Greenwood
Paul Pogba
Fred
Raphaël Varane
Jadon Sancho
Neman

In [20]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [21]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [22]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [23]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [24]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Antonio Rüdiger,2021-08-14,Chelsea,Premier League,1,0,0,94,8,...,0.0,0,0,77,84,91.7,77,0,0,Antonio Rüdiger
1,2021-2022,Antonio Rüdiger,2021-08-22,Chelsea,Premier League,2,0,0,71,6,...,0.0,0,0,56,66,84.8,50,0,0,Antonio Rüdiger
2,2021-2022,Antonio Rüdiger,2021-08-28,Chelsea,Premier League,3,0,0,44,6,...,0.0,0,0,19,30,63.3,17,0,0,Antonio Rüdiger
3,2021-2022,Antonio Rüdiger,2021-09-11,Chelsea,Premier League,4,2,0,54,5,...,0.0,1,0,38,43,88.4,27,0,0,Antonio Rüdiger
4,2021-2022,Antonio Rüdiger,2021-09-19,Chelsea,Premier League,5,2,2,61,5,...,0.1,5,1,48,53,90.6,41,0,0,Antonio Rüdiger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,2021-2022,Bali Mumba,2021-08-21,Norwich,Premier League,2,0,0,19,12,...,0.0,0,0,7,10,70.0,9,0,1,Bali Mumba
1391,2021-2022,Adam Idah,2021-08-14,Norwich,Premier League,1,1,1,6,5,...,0.0,0,0,3,5,60.0,2,0,0,Adam Idah
1392,2021-2022,Adam Idah,2021-08-28,Norwich,Premier League,3,1,0,6,2,...,0.0,0,0,4,4,100.0,5,0,1,Adam Idah
1393,2021-2022,Adam Idah,2021-09-11,Norwich,Premier League,4,0,0,3,2,...,0.2,1,0,1,2,50.0,2,0,0,Adam Idah


In [25]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [26]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

['Asmir Begovic']

In [27]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [36]:
# any additional corrections
# match_list['Asmir Begović'] = 'Asmir Begović'
# match_list['Emerson'] = 'Emerson Aparecido Leite de Souza Junior'
# match_list['Mads Roerslev'] = 'Mads Roerslev Rasmussen'
# match_list['Odsonne Édouard'] = 'Odsonne Edouard'
# match_list['Hwang Hee-chan'] = 'Hee-Chan Hwang'

In [37]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [38]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

[]

In [39]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [40]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [41]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [42]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [43]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [44]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [45]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [46]:
# add crowds field
df_latest_further['crowds'] = 1

In [47]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [48]:
# df_latest_further[df_latest_further['minutes'] > 0][df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()]

In [49]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [50]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
114999,Harry Kane,1,4,0,Tottenham Hotspur,Manchester City,1.560569,2.349720,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115556,Harry Kane,2,4,18,Tottenham Hotspur,Wolverhampton Wanderers,1.558729,0.858519,False,0,...,0.1,2,0,7,10,70,7,0,0,1
116128,Harry Kane,3,4,90,Tottenham Hotspur,Watford,1.537334,0.295470,True,1,...,0.3,3,0,21,26,80.8,30,2,3,1
116711,Harry Kane,4,4,90,Tottenham Hotspur,Crystal Palace,1.547840,0.531751,False,2,...,0,1,0,16,23,69.6,26,2,3,1
117311,Harry Kane,5,4,90,Tottenham Hotspur,Chelsea,1.550871,1.963620,True,2,...,0.0,1,0,24,31,77.4,29,1,2,1


In [51]:
df_train_new.tail(7)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
117696,Yerson Mosquera Valdelamar,5,2,0,Wolverhampton Wanderers,Brentford,0.870223,0.370362,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
117697,Yoane Wissa,5,3,0,Brentford,Wolverhampton Wanderers,0.370362,0.870223,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
117698,Youri Tielemans,5,3,90,Leicester City,Brighton and Hove Albion,1.224008,0.552039,False,5,...,0.9,4,1,55,67,82.1,60,0,1,1
117699,Yves Bissouma,5,3,78,Brighton and Hove Albion,Leicester City,0.552039,1.224008,True,2,...,0.0,0,0,19,23,82.6,13,0,0,1
117700,Zack Steffen,5,1,0,Manchester City,Southampton,2.336319,0.536908,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
117701,Álvaro Fernández,5,1,0,Brentford,Wolverhampton Wanderers,0.370362,0.870223,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
117702,Çaglar Söyüncü,5,2,90,Leicester City,Brighton and Hove Albion,1.224008,0.552039,False,1,...,0.0,0,0,60,67,89.6,52,0,0,1


In [52]:
df_latest_further.shape

(606, 57)

In [53]:
df_train_new.shape

(117703, 59)

In [54]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [55]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [56]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
50,Chelsea,Manchester City,2021-09-25T11:30:00Z,6,0
51,Manchester United,Aston Villa,2021-09-25T11:30:00Z,6,1
52,Everton,Norwich,2021-09-25T14:00:00Z,6,2
53,Leeds,West Ham United,2021-09-25T14:00:00Z,6,3
54,Leicester City,Burnley,2021-09-25T14:00:00Z,6,4
55,Watford,Newcastle United,2021-09-25T14:00:00Z,6,5
56,Brentford,Liverpool,2021-09-25T16:30:00Z,6,6
57,Southampton,Wolverhampton Wanderers,2021-09-26T13:00:00Z,6,7
58,Arsenal,Tottenham Hotspur,2021-09-26T15:30:00Z,6,8
59,Crystal Palace,Brighton and Hove Albion,2021-09-27T19:00:00Z,6,9


In [57]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [58]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,8,43
1,6,1,Manchester United,Aston Villa,2021-09-25T11:30:00Z,1,7
2,6,2,Everton,Norwich,2021-09-25T14:00:00Z,11,45
3,6,3,Leeds,West Ham United,2021-09-25T14:00:00Z,2,21
4,6,4,Leicester City,Burnley,2021-09-25T14:00:00Z,13,90


In [59]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [60]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
1,Rúnar Alex Rúnarsson,0,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,0,3,3,63,3,Willian
8,Héctor Bellerín,0,2,9,48,3,Bellerín
17,Lucas Torreira,0,3,18,45,3,Torreira
22,Konstantinos Mavropanos,0,2,24,45,3,Mavropanos
...,...,...,...,...,...,...,...
594,Andreas Söndergaard,0,1,444,40,39,Söndergaard
596,Yerson Mosquera Valdelamar,0,2,446,45,39,Mosquera
599,Rayan Ait Nouri,0,2,470,43,39,Ait Nouri
601,Patrick Cutrone,0,4,523,50,39,Cutrone


In [61]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [62]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [63]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,6,0,Manchester City,Chelsea,2021-09-25T11:30:00Z,Jack Grealish,3,80,None,Grealish,False
1,6,0,Manchester City,Chelsea,2021-09-25T11:30:00Z,Kyle Walker,2,55,None,Walker,False
2,6,0,Manchester City,Chelsea,2021-09-25T11:30:00Z,Ilkay Gündogan,3,73,50,Gündogan,False
3,6,0,Manchester City,Chelsea,2021-09-25T11:30:00Z,Kevin De Bruyne,3,119,100,De Bruyne,False
4,6,0,Manchester City,Chelsea,2021-09-25T11:30:00Z,John Stones,2,53,75,Stones,False


In [64]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,César Azpilicueta,2,60,None,Azpilicueta,True
1,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Olivier Giroud,4,65,0,Giroud,True
2,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Thiago Emiliano da Silva,2,54,None,Thiago Silva,True
3,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Marcos Alonso,2,58,None,Alonso,True
4,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Jorge Luiz Frello Filho,3,59,None,Jorginho,True


In [65]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [66]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [67]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,César Azpilicueta,2,6.0,1.0,Azpilicueta,True,1.963620,2.336319,2122
1,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Olivier Giroud,4,6.5,0.0,Giroud,True,1.963620,2.336319,2122
2,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Thiago Emiliano da Silva,2,5.4,1.0,Thiago Silva,True,1.963620,2.336319,2122
3,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Marcos Alonso,2,5.8,1.0,Alonso,True,1.963620,2.336319,2122
4,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Jorge Luiz Frello Filho,3,5.9,1.0,Jorginho,True,1.963620,2.336319,2122
5,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Ross Barkley,3,5.3,1.0,Barkley,True,1.963620,2.336319,2122
6,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Mateo Kovacic,3,5.1,1.0,Kovacic,True,1.963620,2.336319,2122
7,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Michy Batshuayi,4,5.5,0.0,Batshuayi,True,1.963620,2.336319,2122
8,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Antonio Rüdiger,2,5.6,1.0,Rüdiger,True,1.963620,2.336319,2122
9,6,0,Chelsea,Manchester City,2021-09-25T11:30:00Z,Kepa Arrizabalaga,1,4.9,1.0,Arrizabalaga,True,1.963620,2.336319,2122


In [68]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')